# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper

This notebook is supposed to be run after asborption features have been identified using pyigm_igm guesses

Install: https://github.com/pyigm/pyigm

Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [95]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

In [97]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamandJessIDs_Aug23_2023.json'

# normalization file that you want to use
norm = 'J1235+1123_nbin3_method1_jwnorm.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/J0246-3007_nbin3_method1_jwnorm.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [98]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [99]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

`ftol` termination condition is satisfied.
Function evaluations 5, initial cost 9.1935e+00, final cost 7.8792e+00, first-order optimality 3.56e-04.

Fit results: 

977.02	 0.528430	 13.804	 39.472	 1.3503
 	  	  	 0.042	 4.311	 3.035 


Reduced chi-squared: 0.463481
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_0.txt
Line parameters written to:
component_groups/input_group_0.VP
Wrote spectrum to component_groups/input_group_0_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_0_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 7, initial cost 1.7354e+01, final cost 1.6399e+01, first-order optimality 2.67e-03.

Fit results: 

1526.71	 0.000050	 14.451	 32.379	 -2.6333
 	  	  	 0.051	 3.574	 2.518 


Reduced chi-squared: 0.840965
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_1.txt
Line parameters written to:
component_groups/input_group_1.VP
Wrote spectrum to component_groups/input_group_1_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_1_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 15, initial cost 5.6202e+01, final cost 5.1396e+01, first-order optimality 1.46e-03.

Fit results: 

1548.2	 0.004260	 14.143	 55.600	 3.9863
 	  	  	 0.046	 7.584	 5.183 

1550.78	 0.004260	 14.143	 55.600	 3.9863
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.004710	 13.709	 15.365	 -4.4806
 	  	  	 0.104	 6.09	 3.752 

1550.78	 0.004710	 13.709	 15.365	 -4.4806
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.901687
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_3.txt
Line parameters written to:
component_groups/input_group_3.VP
Wrote spectrum to component_groups/input_group_3_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_3_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 7, initial cost 3.9686e+01, final cost 3.2144e+01, first-order optimality 3.35e-04.

Fit results: 

1548.2	 0.000090	 13.960	 31.081	 -11.23
 	  	  	 0.05	 4.952	 3.307 

1550.78	 0.000090	 13.960	 31.081	 -11.23
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.931706
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_2.txt
Line parameters written to:
component_groups/input_group_2.VP
Wrote spectrum to component_groups/input_group_2_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_2_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 12, initial cost 9.9015e+00, final cost 7.7799e+00, first-order optimality 9.70e-04.

Fit results: 

1670.79	 0.000050	 13.185	 45.209	 -7.3335
 	  	  	 0.076	 8.864	 6.439 


Reduced chi-squared: 0.420537
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_6.txt
Line parameters written to:
component_groups/input_group_6.VP
Wrote spectrum to component_groups/input_group_6_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_6_inspect.fits
There are no valid pixels to fit in the region. Moving on.
`ftol` termination condition is satisfied.
Function evaluations 10, initial cost 3.3738e+01, final cost 3.2722e+01, first-order optimality 1.10e-03.

Fit results: 

1560.31	 0.002370	 14.209	 31.072	 8.4211
 	  	  	 0.107	 7.828	 5.669 

1656.93	 0.002370	 14.209	 31.072	 8.4211
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.908939
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_4.txt
Line parameters written to:
component_groups/input_group_4.VP
Wrote spectrum to component_groups/input_group_4_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_4_inspect.fits

VPmeasure: 6/7 fits converged, 0/7 failed (see log for details).

VPmeasure: concatenating individual outputs and creating figures for inspection.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(),

Wrote spectrum to FitInspection.fits
VPmeasure: Done.


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


In [100]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J0246_rereduc' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [101]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

['component_groups/input_group_2.VP',
 'component_groups/input_group_6.VP',
 'component_groups/input_group_3.VP',
 'component_groups/input_group_4.VP',
 'component_groups/input_group_0.VP',
 'component_groups/input_group_1.VP']

In [102]:
#Create a pandas data frame
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

,specfile,restwave,zsys,col,sigcol,bval,sigbval,vel,sigvel,nflag,...,vlim1,vlim2,wobs1,wobs2,pix1,pix2,z_comp,trans,rely,comment
0,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1548.2040,0.00009,13.960,0.050,31.081,4.952,-11.230,3.307,2,...,-56.3752,56.3752,1548.045994,1548.628266,4498,4514,0.00005,C IV,a,--
1,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1550.7810,0.00009,13.960,0.000,31.081,0.000,-11.230,0.000,2,...,-56.3752,56.3752,1550.622731,1551.205972,4569,4585,0.00005,C IV,a,--
2,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1670.7886,0.00005,13.185,0.076,45.209,8.864,-7.333,6.439,2,...,-63.1403,63.1403,1670.520015,1671.223796,7836,7856,0.00003,AlII,a,--
3,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1548.2040,0.00426,14.143,0.046,55.600,7.584,3.986,5.183,2,...,-92.5266,92.5266,1554.314474,1555.270135,4669,4695,0.00427,C IV,b,--
4,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1550.7810,0.00426,14.143,0.000,55.600,0.000,3.986,0.000,2,...,-92.5266,92.5266,1556.901645,1557.858897,4740,4766,0.00427,C IV,b,--
5,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1548.2040,0.00471,13.709,0.104,15.365,6.090,-4.481,3.752,3,...,-54.5020,54.5020,1555.220772,1555.783696,4694,4709,0.00470,C IV,b,--
6,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1550.7810,0.00471,13.709,0.000,15.365,0.000,-4.481,0.000,3,...,-54.5020,54.5020,1557.809451,1558.373312,4764,4780,0.00470,C IV,b,--
7,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1560.3092,0.00237,14.209,0.107,31.072,7.828,8.421,5.669,2,...,-58.6302,58.6302,1563.701641,1564.311938,4925,4942,0.00240,C I,c,--
8,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,1656.9284,0.00237,14.209,0.000,31.072,0.000,8.421,0.000,2,...,-58.6302,58.6302,1660.530911,1661.179000,7564,7582,0.00240,C I,c,--
9,/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246...,977.0201,0.52843,13.804,0.042,39.472,4.311,1.350,3.035,2,...,-65.3953,65.3953,1493.093641,1493.519886,3001,3012,0.52844,C III,c,--


In [103]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [104]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/J0246_rereduc/component_groups'

In [39]:
parent_path

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/'

In [41]:
path

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/J1159_rereduc'

In [105]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)

Moved: _VP_log.dat
Moved: all_VP.txt
Moved: compiledVPoutputs.dat
Moved: FitInspection.fits
Moved: FitInspection.pdf
